# AI4I MLOps for Predictive Maintenance

##  Issues with Jupyter Notebook Space vs Sagemaker container
sagemaker-scikit-learn-container requirements are defined in https://github.com/aws/sagemaker-scikit-learn-container/blob/master/requirements.txt

we are interested in the following from the requirements:
```python
pandas==1.1.3
scikit-learn==1.2.1
```
One of the challenges we encountered is that our sagemaker jupyter notebook is running scikit-learn 1.6.1 as we used the default space
https://github.com/aws/sagemaker-distribution/blob/main/build_artifacts/v3/v3.3/v3.3.2/RELEASE.md
with scikit-learn == 1.6.1 

This cause our manually serialized model failed to be loaded by sagemaker due to the changes between scikit-learn==1.6.1 (model) and scikit-learn==1.2.1 (sagemaker endpoint container)

We were unable to solve this and failed to create endpoint manually

We then try to create new space to run our jupyter notebook using much older sagemaker distribution

The only one that should work is 
https://github.com/aws/sagemaker-distribution/blob/main/build_artifacts/v0/v0.1/v0.1.2/RELEASE.md
with scikit-learn == 1.2.2 

However, AWS no longer provide Space with this image.

Hence we are not able to manually setup sagemaker endpoint unless we can train using sagemaker pipeline instead of our jupyter notebook

## Scripts

Here we write our scripts into files

### Requirements file

In [1]:
%%writefile requirements.txt
boto3==1.28.57
botocore==1.31.85
pandas==1.5.3
numpy==1.24.3
mlflow
sagemaker-mlflow
scikit-learn==1.2.2
joblib

Overwriting requirements.txt


### Preprocessing

We write our preprocessing step as a separate script file.

We then import the preprocessing function to our notebook.

This allows us to re-use it in our pipeline definition later.

One drawback is that we need to restart our kernel whenever we update this script

In [2]:
%%writefile preprocess.py
## This file is created once during manual setup 
import os
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
def unknown_fail_check(row): return ((row['Machine failure'] == 1)
                                     & (row['RNF'] == 0)
                                     & (row['HDF'] == 0)
                                     & (row['TWF'] == 0)
                                     & (row['PWF'] == 0)
                                     & (row['OSF'] == 0))

def pass_yet_fail_check(row): return (row['Machine failure'] == 0) & ((row['RNF'] == 1)
                                                                     | (row['HDF'] == 1)
                                                                     | (row['TWF'] == 1)
                                                                     | (row['PWF'] == 1)
                                                                     | (row['OSF'] == 1))
def preprocessing(df):
    print("# Preprocessing")
    df['Type'] = df['Type'].astype('category')
    type_mapping = {'L': 0, 'M': 1, 'H': 2}
    df['Type'] = df['Type'].map(type_mapping).astype('int')
    print(" Type  Unique Values after encoding: ", df['Type'].unique())
    df.drop(columns=['UDI', 'Product ID'], inplace=True)
    print(f"shape of data after dropping columns {df.shape}")
    df.columns = [col.replace("[","(").replace("]",")") for col in df.columns.values]
    print("DF columns after clean up", df.columns)
    print("## Handle Duplicates") 
    # our original dataset does not have duplicates
    # However, there is no guarantee that production/new data is free of duplicates
    duplicated_row_count = df.duplicated().sum()
    total_row_count = df.shape[0]
    duplicated_row_percentage = (duplicated_row_count/total_row_count*100)
    print(f"Total rows count: {total_row_count}")
    print(f"Duplicated rows count: {duplicated_row_count}")
    print(f"Duplicated rows percentage: {duplicated_row_percentage}")
    df.drop_duplicates(inplace=True)
    print("After removing duplicates rows count:", df.shape[0])
    print("## Handle NULL") 
    print("number of null values : ", df.isnull().sum().sum())
    df.dropna(inplace=True)
    print("After removing null rows count:", df.shape[0])
    # declare our target labels columns
    labels = ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
    passed_although_failed = df[pass_yet_fail_check(df)]
    print(
        f"Number of samples that passed although failed: {len(passed_although_failed)}")
    passed_although_failed.loc[:, labels].head(10)
    df['Machine failure'] = np.where(
        pass_yet_fail_check(df), 1, df['Machine failure'])
    passed_although_failed = df[pass_yet_fail_check(df)]
    print(
        f"Number of samples that passed although failed after fix: {len(passed_although_failed)}")

    print(f"Number of machine failures: {df['Machine failure'].sum()}")
    unknown_failures = df[unknown_fail_check(df)]
    print(
        f"Number of failures due to unknown reasons: {len(unknown_failures)}")
    unknown_failures.loc[:, labels].head(10)
    df['Machine failure'] = np.where(
        unknown_fail_check(df), 0, df['Machine failure'])
    unknown_failures = df[unknown_fail_check(df)]
    print(
        f"Number of failures due to unknown reasons after fix: {len(unknown_failures)}")
    print("## Add Features") 
    df['Strain (minNm)'] = df['Tool wear (min)'] * df['Torque (Nm)'] 
    df['Power (W)'] = df['Rotational speed (rpm)'] * df['Torque (Nm)'] * 2 * np.pi / 60
    df['Temperature Difference (K)'] = df['Process temperature (K)'] - df['Air temperature (K)']
    print("columns after new feature : ",df.columns)
    print("## Drop Redudant Features")
    df.drop(columns=['Torque (Nm)', 'Process temperature (K)', 'Air temperature (K)'], inplace=True)
    print("columns after drop :",df.columns)
    print("## Convert Int to Float for MLFlow")
    for col in df.columns:
        if df[col].dtype == np.int64 or df[col].dtype == np.int32: # Check for common integer types
            df[col] = df[col].astype(np.float64)
    print("# Splitting into train/test...")
    X = df.drop(columns=labels)
    y = df[labels]
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42,  stratify=y['Machine failure']) 
    train=pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    print("# Processing Done")
    return train, test

if __name__ == "__main__":
    # The pipeline will pass arguments to this script.
    # The argument will be used to pass the S3 path of our data.
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-path", type=str, help="path containing data.csv")
    parser.add_argument("--output-train-path", type=str, help="Output directory for train.csv")
    parser.add_argument("--output-test-path", type=str, help="Output directory for test.csv")
    args = parser.parse_args()

    input_path = args.input_path or "/opt/ml/processing/input"
    output_train_path = args.output_train_path or "/opt/ml/processing/train"
    output_test_path = args.output_test_path or "/opt/ml/processing/test"
    print(f"--- Starting Processing Job ---")
    print(f"Input path: {input_path}")
    print(f"Output train path: {output_train_path}")
    print(f"Output test path: {output_test_path}")
    # Load the dataset
    print(f"Loading data from {input_path}/data.csv")
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input path {input_path} does not exist.")
    if not os.path.exists(os.path.join(input_path, "data.csv")):
        raise FileNotFoundError(f"Data file not found in {input_path}. Please check the path.")
    # Read the CSV file 
    data_path = os.path.join(input_path, "data.csv")
    df = pd.read_csv(data_path) 
    # Preprocess
    train, test = preprocessing(df)
    os.makedirs(output_train_path, exist_ok=True)
    os.makedirs(output_test_path, exist_ok=True)
    print(f"Saving train data to {output_train_path}/train.csv")
    train.to_csv(os.path.join(output_train_path, "train.csv"), index=False)
    print(f"Saving test data to {output_test_path}/test.csv")
    test.to_csv(os.path.join(output_test_path, "test.csv"), index=False)
    print("--- Processing Job Completed ---")


Overwriting preprocess.py


### Training

In [3]:
%%writefile train.py
# # Ensure MLflow is installed
import sys
import subprocess
import pandas as pd
try:
    import mlflow
    import sagemaker_mlflow
except ImportError:
    print("Installing MLflow...")
    subprocess.check_call([sys.executable, "-m", "pip", "install",
                            "boto3==1.37.1", "botocore==1.37.1", "s3transfer",
                            "mlflow==2.22.0", "sagemaker-mlflow==0.1.0"])
import mlflow
import sagemaker_mlflow
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

def train(df, max_depth=10):
    # declare our target labels columns
    labels = ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
    X = df.drop(columns=labels)
    y = df[labels]
    model = RandomForestClassifier(random_state=42, max_depth=max_depth)
    model = MultiOutputClassifier(model)
    model.fit(X, y)
    y_pred = model.predict(X)
    y_pred = pd.DataFrame(y_pred, columns=y.columns)
    y_pred_omf = y_pred["Machine failure"]
    y_omf = y["Machine failure"]
    f2 = fbeta_score(y_omf, y_pred_omf, beta=2)
    recall = recall_score(y_omf, y_pred_omf)
    precision = precision_score(y_omf, y_pred_omf, zero_division=0)
    accuracy = accuracy_score(y_omf, y_pred_omf)
    return {
        "model": model,
        "f2": f2,
        "recall": recall,
        "precision": precision,
        "accuracy": accuracy,
        "input_example": X.head(5)
    }


def run_experiment(df, experiment_name, run_name=None, model_name="model",
                   max_depth=10):
    print("Starting experiment ", experiment_name, " with run name ", run_name)
    run_id = None
    # Start an MLflow run
    # Use the experiment name and run name to organize runs
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=run_name) as run:
        run_id = run.info.run_id
        print(f"\tMLflow Run ID : {run_id}")
        print(f"\tRunning experiment: {experiment_name}, Run Name: {run_name}")
        # Train the model
        # Provide the first 5 rows of the training data as an example
        result = train(df, max_depth=max_depth)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("f2", result["f2"])
        mlflow.log_metric("accuracy", result["accuracy"])
        mlflow.log_metric("recall", result["recall"])
        mlflow.log_metric("precision", result["precision"])
        model = result["model"]
       # Train the model
        # Provide the first 5 rows of the training data as an example
        input_example = result["input_example"]
        # Get the run ID for later use
        mlflow.sklearn.log_model(
            sk_model=model, artifact_path=model_name, input_example=input_example)
        print("\tFinished: experiment ", experiment_name,
              " with run name ", run_name)
    return model, run_id, result["accuracy"], result["f2"]


if __name__ == "__main__":


    # import mlflow
    # import sagemaker_mlflow
    import mlflow.sklearn
    import os
    import argparse
    import pandas as pd
    import joblib
    import glob

    parser = argparse.ArgumentParser()
    parser.add_argument("--tracking_server_arn", type=str, required=True)
    parser.add_argument("--experiment_name", type=str, default="Default")
    parser.add_argument("--model_output_path", type=str,
                        default="/opt/ml/model")
    parser.add_argument("--max_depth", type=int, default=5)
    args, _ = parser.parse_known_args()

    # Load training data
    train_path = glob.glob("/opt/ml/input/data/train/*.csv")[0]
    df = pd.read_csv(train_path)
    # Set up MLflow
    mlflow.set_tracking_uri(args.tracking_server_arn)
    the_model, run_id, accuracy_score, f2_score = run_experiment(df=df,
                                                             experiment_name=args.experiment_name,
                                                             run_name="run_name",
                                                             model_name="model",
                                                             max_depth=args.max_depth
                                                             )
    os.makedirs(args.model_output_path, exist_ok=True)
    joblib.dump(the_model, os.path.join(args.model_output_path, "model.joblib"))
    with open(os.path.join(args.model_output_path, "run_id.txt"), "w") as f:
        f.write(run_id)

    print(
        f"Training complete. F2:{f2_score:.4f} Accuracy: {accuracy_score:.4f}")
    print(f"MLflow Run ID: {run_id}")

Overwriting train.py


### Evaluation script

Here we create evaluation script

In [4]:
%%writefile evaluate.py
import argparse
import pandas as pd
from sklearn.metrics import fbeta_score
import joblib
import os
import json
import boto3
import tarfile

if __name__ == "__main__":
    # --- Parse Arguments ---
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-path", type=str, required=True, help="Path to the directory containing the model.tar.gz file.")
    parser.add_argument("--test-path", type=str, required=True, help="Path to the directory containing test.csv.")
    parser.add_argument("--output-path", type=str, required=True, help="Path to save the evaluation.json report.")
    parser.add_argument("--model-package-group-name", type=str, required=True, help="Name of the SageMaker Model Package Group.")
    parser.add_argument("--region", type=str, required=True, help="The AWS region for creating the boto3 client.")
    args = parser.parse_args()

    # --- Extract and Load Model ---
    # SageMaker packages models in a .tar.gz file. We need to extract it first.
    model_archive_path = os.path.join(args.model_path, 'model.tar.gz')
    print(f"Extracting model from archive: {model_archive_path}")
    with tarfile.open(model_archive_path, "r:gz") as tar:
        tar.extractall(path=args.model_path)

    # Load the model using joblib
    model_file_path = os.path.join(args.model_path, "model.joblib")
    if not os.path.exists(model_file_path):
        raise FileNotFoundError(f"Model file 'model.joblib' not found after extraction in: {args.model_path}")
    
    print(f"Loading model from: {model_file_path}")
    model = joblib.load(model_file_path)

    # --- Prepare Data and Evaluate ---
    test_file_path = os.path.join(args.test_path, "test.csv")
    if not os.path.exists(test_file_path):
        raise FileNotFoundError(f"Test data not found: {test_file_path}")
    
    test_df = pd.read_csv(test_file_path)
    labels = ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
    X_test = test_df.drop(columns=labels)
    y_test= test_df[labels]
    
    print("Running predictions on the test dataset.")
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
    y_pred_omf = y_pred["Machine failure"]
    y_test_omf = y_test["Machine failure"]
    f2_score = fbeta_score(y_test_omf, y_pred_omf, beta=2)
    report = {"f2": f2_score}
    print(f"Calculated f2: {f2_score:.4f}")

    # --- Check for Existing Baseline Model in SageMaker Model Registry ---
    print(f"Checking for baseline model in region: {args.region}")
    sagemaker_client = boto3.client("sagemaker", region_name=args.region)
    try:
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=args.model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            SortOrder="Descending",
            MaxResults=1,
        )
        # If the list is not empty, an approved model already exists
        report["baseline_exists"] = len(response["ModelPackageSummaryList"]) > 0
        if report["baseline_exists"]:
            print(f"An approved baseline model was found in '{args.model_package_group_name}'.")
        else:
             print(f"No approved baseline model was found in '{args.model_package_group_name}'.")

    except sagemaker_client.exceptions.ClientError as e:
        # If the ModelPackageGroup doesn't exist, there is no baseline
        if "ResourceNotFound" in str(e):
            report["baseline_exists"] = False
            print(f"Model Package Group '{args.model_package_group_name}' not found. Assuming no baseline exists.")
        else:
            raise

    # --- Write Final Report ---
    os.makedirs(args.output_path, exist_ok=True)
    report_path = os.path.join(args.output_path, "evaluation.json")
    with open(report_path, "w") as f:
        json.dump(report, f, indent=4)
        
    print(f"✅ Evaluation complete. Report written to: {report_path}")
    print("Evaluation Report:")
    print(json.dumps(report, indent=4))

Overwriting evaluate.py


### Deployment Script

In [5]:
%%writefile deploy.py
import subprocess
import sys

# --- Install required packages ---
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "boto3==1.28.57", "botocore==1.31.57", "numpy==1.24.1", "sagemaker" ])

# Ensure sagemaker SDK is installed before importing
try:
    import sagemaker
except ImportError:
    print("sagemaker SDK not found. Installing now...")
    install("sagemaker")
    import sagemaker

import argparse
import sagemaker
import boto3
from sagemaker.model import ModelPackage

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    # Accept the registered model's ARN instead of the S3 data path
    parser.add_argument("--model-package-arn", type=str, required=True)
    parser.add_argument("--role", type=str, required=True)
    parser.add_argument("--endpoint-name", type=str, required=True)
    parser.add_argument("--region", type=str, required=True)
    args = parser.parse_args()

    boto_session = boto3.Session(region_name=args.region)
    sagemaker_session = sagemaker.Session(boto_session=boto_session)

    # Create a SageMaker Model object directly from the Model Package ARN
    model = ModelPackage(
        model_package_arn=args.model_package_arn,
        role=args.role,
        sagemaker_session=sagemaker_session,
    )

    # Deploy the model to an endpoint
    print(f"Deploying registered model from ARN to endpoint: {args.endpoint_name}")
    model.deploy(
        initial_instance_count=1,
        instance_type="ml.t2.medium",
        endpoint_name=args.endpoint_name,
        # Update endpoint if it already exists
        update_endpoint=True
    )
    print("Deployment complete.")


Overwriting deploy.py


## Manual Run

We perform manual run to validate our cloud infrastructure and scripts
before we create our pipeline

### Dependencies

In [6]:
import sklearn
import joblib
import sagemaker
print(sagemaker.__version__)
print(joblib.__version__)
print(sklearn.__version__)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
2.245.0
1.5.1
1.6.1


###  Load Data Set

In [7]:
import pandas as pd
df = pd.read_csv("ai4i2020.csv")
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.info()}")
print(f"Columns: {df.columns.tolist()}")
print(f"First 3 rows:\n{df.head(3)}") 

Dataset shape: (10000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Machine failure          10000 non-null  int64  
 9   TWF                      10000 non-null  int64  
 10  HDF                      10000 non-null  int64  
 11  PWF                      10000 non-null  int64  
 12  OSF                      10000 non-null  int64  
 13  RNF                      10000 non-null  int64  
d

### Preprocessing Manual Run

Let's import preprocessing function from preprocess.py and run it

In [8]:
from preprocess import preprocessing 
# since we write the preprocess script we can now import it into our notebook
train, test= preprocessing(df)

# Preprocessing
 Type  Unique Values after encoding:  [1 0 2]
shape of data after dropping columns (10000, 12)
DF columns after clean up Index(['Type', 'Air temperature (K)', 'Process temperature (K)',
       'Rotational speed (rpm)', 'Torque (Nm)', 'Tool wear (min)',
       'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')
## Handle Duplicates
Total rows count: 10000
Duplicated rows count: 0
Duplicated rows percentage: 0.0
After removing duplicates rows count: 10000
## Handle NULL
number of null values :  0
After removing null rows count: 10000
Number of samples that passed although failed: 18
Number of samples that passed although failed after fix: 0
Number of machine failures: 357
Number of failures due to unknown reasons: 9
Number of failures due to unknown reasons after fix: 0
## Add Features
columns after new feature :  Index(['Type', 'Air temperature (K)', 'Process temperature (K)',
       'Rotational speed (rpm)', 'Torque (Nm)', 'Tool wear (min)',
   

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Type                        10000 non-null  float64
 1   Rotational speed (rpm)      10000 non-null  float64
 2   Tool wear (min)             10000 non-null  float64
 3   Machine failure             10000 non-null  float64
 4   TWF                         10000 non-null  float64
 5   HDF                         10000 non-null  float64
 6   PWF                         10000 non-null  float64
 7   OSF                         10000 non-null  float64
 8   RNF                         10000 non-null  float64
 9   Strain (minNm)              10000 non-null  float64
 10  Power (W)                   10000 non-null  float64
 11  Temperature Difference (K)  10000 non-null  float64
dtypes: float64(12)
memory usage: 937.6 KB


In [10]:
from train import run_experiment
import mlflow
mlflow.set_tracking_uri("mlruns")
run_experiment(df, "experiment","run", "model")

Starting experiment  experiment  with run name  run
	MLflow Run ID : 7c9a2d8ae3f54d3a9c63394be4791a97
	Running experiment: experiment, Run Name: run
	Finished: experiment  experiment  with run name  run


(MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                        random_state=42)),
 '7c9a2d8ae3f54d3a9c63394be4791a97',
 0.9945,
 0.8694362017804155)

###  Configuration

In [11]:
# ----------------------
base_folder = 'ai4i'      # e.g., 'users/my-name'
experiment_name = "ai4i-Experiment"  # e.g., 'my-experiment'
model_name = "ai4i-model"  # e.g., 'my-model'
tracking_server_name = "Team16-MLFlow"
bucket_name="iti113-team16-bucket" # s3://iti113-team16-bucket/ai4i/mlflow/

### Create SageMaker and S3 Clients

In [12]:
import sagemaker
import boto3
import mlflow
from sklearn.metrics import classification_report

sagemaker_client = None
s3_client = None
try:
    sagemaker_session = sagemaker.Session()
    sagemaker_client = boto3.client("sagemaker")
    s3_bucket = sagemaker_session.default_bucket()
    s3_client = boto3.client('s3')
    s3_data_key=f"{base_folder}/data/v1/data.csv"
    s3_data_path = f"s3://{bucket_name}/{s3_data_key}"
    s3_data_dir_uri = f"s3://{bucket_name}/{base_folder}/data/v1"
    print(f"Your datasets will be versioned inside: {s3_data_path}")
except Exception as e:
    print(f"Error initializing SageMaker session or S3 client: {e}")
    s3_data_path = None
# minimize traceback in the output as we are not interested in the details
if not sagemaker_client or not s3_client:
    raise Exception("Failed to initialize SageMaker session or S3 client.")


Your datasets will be versioned inside: s3://iti113-team16-bucket/ai4i/data/v1/data.csv


### Connect to Tracking Server


In [13]:
mlflow_tracking_server_arn = None
try:
    response = sagemaker_client.describe_mlflow_tracking_server(
        TrackingServerName=tracking_server_name
    )
    # ARN of MLflow Tracking Server
    mlflow_tracking_server_arn = response['TrackingServerArn']
    print(f"Found MLflow Tracking Server ARN: {mlflow_tracking_server_arn}")
except Exception as e:
    print(f"Could not find tracking server: {e}")
    mlflow_tracking_server_arn = None

# minimize traceback in the output as we are not interested in the details
if not mlflow_tracking_server_arn:
    raise Exception("Failed to find MLflow Tracking Server.")

# IAM role for SageMaker execution
role = sagemaker.get_execution_role()

print(f"S3 Bucket: {s3_data_path}")
print(f"SageMaker Role ARN: {role}")
print(f"MLflow Tracking Server ARN: {mlflow_tracking_server_arn}")

# Connect to the MLflow Tracking Server
# Set the MLflow tracking URI to managed server
if mlflow_tracking_server_arn:
    mlflow.set_tracking_uri(mlflow_tracking_server_arn)
    print("MLflow tracking URI set successfully.")

# Define an experiment name. If it doesn't exist, MLflow creates it.
mlflow.set_experiment(experiment_name)

print(f"MLflow tracking URI set to: {mlflow.get_tracking_uri()}")
print(f"MLflow experiment set to: '{experiment_name}'")

Found MLflow Tracking Server ARN: arn:aws:sagemaker:ap-southeast-1:837028399719:mlflow-tracking-server/Team16-MLFlow
S3 Bucket: s3://iti113-team16-bucket/ai4i/data/v1/data.csv
SageMaker Role ARN: arn:aws:iam::837028399719:role/iti113-team16-sagemaker-iti113-team16-domain-iti113-team16-Role
MLflow Tracking Server ARN: arn:aws:sagemaker:ap-southeast-1:837028399719:mlflow-tracking-server/Team16-MLFlow
MLflow tracking URI set successfully.
MLflow tracking URI set to: arn:aws:sagemaker:ap-southeast-1:837028399719:mlflow-tracking-server/Team16-MLFlow
MLflow experiment set to: 'ai4i-Experiment'


### Run Experiments and Track


Let's run our experiments while varying the hyperparameter

In [14]:
# Run experiments with different max_depth parameters
#
experiments={
    "md4":4,
    "md5":5,
    "md10":10,
    "md100":100
}
results = {}
best_run_id = None
best_run_name = None
best_f2 = 0.0
best_md_param = None
for run_name, md_param in experiments.items():
    model, run_id, accuracy_score, f2_score = run_experiment(df, experiment_name, run_name, model_name, max_depth=md_param)
    results[run_name] = {
        'run_id': run_id,
        'accuracy': accuracy_score,
        'f2': f2_score
    }
    if best_f2 < f2_score:
        best_f2 = f2_score
        best_run_id = run_id
        best_run_name = run_name
        best_md_param = md_param
    elif best_f2 == f2_score:
        print(f"Found another run with same accuracy: {f2_score:.4f}")
        print(f"run {best_run_name} vs run {run_name}")
        if best_md_param is None or md_param < best_md_param:
            # Update the best run if the C parameter is lower
            best_md_param = md_param
            best_run_id = run_id
            best_run_name = run_name
            print(f"\t Updating best run to {run_name} with max_depth={md_param} and f2_score={f2_score:.4f}")
        else:
            print(f"\t Keeping best run {best_run_name} with smaller max_depth to have simpler model")

print(f"Best run: {best_run_name} id: {best_run_id} with f2_score: {best_f2:.4f}")

Starting experiment  ai4i-Experiment  with run name  md4
	MLflow Run ID : cdacf6ef3f384f6ea7626bb430c3d543
	Running experiment: ai4i-Experiment, Run Name: md4
	Finished: experiment  ai4i-Experiment  with run name  md4
🏃 View run md4 at: https://ap-southeast-1.experiments.sagemaker.aws/#/experiments/1/runs/cdacf6ef3f384f6ea7626bb430c3d543
🧪 View experiment at: https://ap-southeast-1.experiments.sagemaker.aws/#/experiments/1
Starting experiment  ai4i-Experiment  with run name  md5
	MLflow Run ID : dfbf88bd27f6479bbe09eace4b9ab14b
	Running experiment: ai4i-Experiment, Run Name: md5
	Finished: experiment  ai4i-Experiment  with run name  md5
🏃 View run md5 at: https://ap-southeast-1.experiments.sagemaker.aws/#/experiments/1/runs/dfbf88bd27f6479bbe09eace4b9ab14b
🧪 View experiment at: https://ap-southeast-1.experiments.sagemaker.aws/#/experiments/1
Starting experiment  ai4i-Experiment  with run name  md10
	MLflow Run ID : d20ce283b2b9493ca8229147b492174b
	Running experiment: ai4i-Experiment, 

### Model Registration

The best model is the one with run_id stored in `best_run_id`
Let us save it to S3

In [15]:
def mlflow_register_model(run_id):
    print(f"Saving model with run ID: {run_id} to S3")
    model_uri = f"runs:/{run_id}/{model_name}"
    print(f"\tRegistering model from URI: {model_uri}")
    # Register the model to the MLflow Model Registry
    reg_model = mlflow.register_model(
        model_uri=model_uri,
        name=model_name
    )
    print(f"\tModel '{model_name}' registered with version: {reg_model.version}")
    return reg_model


registered_model = mlflow_register_model(best_run_id)

Saving model with run ID: cdfae16dd1594fcabc7c39ae78941e8b to S3
	Registering model from URI: runs:/cdfae16dd1594fcabc7c39ae78941e8b/ai4i-model


Registered model 'ai4i-model' already exists. Creating a new version of this model...
2025/08/22 02:52:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ai4i-model, version 8


	Model 'ai4i-model' registered with version: 8


Created version '8' of model 'ai4i-model'.


Check Model has been Registered properly

In [16]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
model_version = client.get_model_version(model_name, registered_model.version)
model_artifact_s3 = model_version.source
model_version_no = model_version.version
model_version_name = model_version.name
print("Model S3 Artifact URI:", model_artifact_s3)
print("Model Version No     :", model_version_no)
print("Model Version Name   :", model_version_name)


Model S3 Artifact URI: s3://iti113-team16-bucket/ai4i/mlflow/1/cdfae16dd1594fcabc7c39ae78941e8b/artifacts/ai4i-model
Model Version No     : 8
Model Version Name   : ai4i-model


In [17]:
import tarfile
import os
import joblib

model_s3_uri = None
def download_model_artifact(model_version_name,model_version_no, model_folder="/tmp/model"):
    """
    Download the model artifact from the MLflow Model Registry.
    """
    artifact_uri=f"models:/{model_version_name}/{model_version_no}"
    print(f"Model artifact {artifact_uri}")
    os.makedirs(model_folder, exist_ok=True)
    mlflow.artifacts.download_artifacts(
        artifact_uri=artifact_uri,
        dst_path=model_folder
    )
    print(f"Model artifact {artifact_uri} downloaded to: {model_folder}")

# Download the model artifact
def create_model_archive(model_folder="/tmp/model", model_tgz_path="/tmp/model.tar.gz"):
    """
    Create a tar.gz archive of the model folder.
    """
    with tarfile.open(model_tgz_path, "w:gz") as tar:
        tar.add(model_folder, arcname=".")
    print(f"Model archive created at: {model_tgz_path}")

def test_load_archive(model_folder="/tmp/model"):
    model=joblib.load(os.path.join(model_folder, "model.pkl"))
    print(model)

def upload_to_s3(local_file, model_version_name, model_version_no):
    """
    Upload a local file to an S3 bucket.
    """
    s3_key = f"{base_folder}/models/{model_version_name}-v{model_version_no}/model.tar.gz"
    print(f"Uploading {local_file} to s3 with key {s3_key}")
    model_s3_uri = None
    try:
        bucket = sagemaker_session.default_bucket() 
        s3_client.upload_file(local_file, bucket, s3_key)
        model_s3_uri = f"s3://{bucket}/{s3_key}"
    except Exception as e:
        print(f"Error uploading to S3: {e}")
    # minimize traceback in the output as we are not interested in the details
    if not model_s3_uri:
        raise Exception("Failed to upload model to S3.")
    print(f"File {local_file} uploaded to {model_s3_uri}")
    return model_s3_uri

# Create a compressed archive of the model folder
model_folder="/tmp/model"
model_tgz_path="/tmp/model.tar.gz"
print("expect s3://iti113-team16-bucket/ai4i/mlflow/1/fc49513036cb442ea9eb764c32509bfa/artifacts/model")
download_model_artifact(model_version_name, model_version_no, model_folder)
create_model_archive(model_folder, model_tgz_path)
test_load_archive(model_folder)
model_s3_uri = upload_to_s3(model_tgz_path, model_version_name, model_version_no)
# Upload to S3
print("✅ Compressed model uploaded to:", model_s3_uri)
    

expect s3://iti113-team16-bucket/ai4i/mlflow/1/fc49513036cb442ea9eb764c32509bfa/artifacts/model
Model artifact models:/ai4i-model/8


Model artifact models:/ai4i-model/8 downloaded to: /tmp/model
Model archive created at: /tmp/model.tar.gz
MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       random_state=42))
Uploading /tmp/model.tar.gz to s3 with key ai4i/models/ai4i-model-v8/model.tar.gz
File /tmp/model.tar.gz uploaded to s3://sagemaker-ap-southeast-1-837028399719/ai4i/models/ai4i-model-v8/model.tar.gz
✅ Compressed model uploaded to: s3://sagemaker-ap-southeast-1-837028399719/ai4i/models/ai4i-model-v8/model.tar.gz


## Pipeline Automation For Preprocessing and Training

### Configuration

In [18]:
# ----------------------
model_name = "ai4i-pipeline-model"  # e.g., 'my-model'
model_package_group_name = "AI4IPipelineModels"
pipeline_name = "AI4IClassifierPipeline"
# ----------------------
base_folder = 'ai4i'      # e.g., 'users/my-name'
experiment_name = "ai4i-Experiment"  # e.g., 'my-experiment'
model_name = "ai4i-model"  # e.g., 'my-model'
tracking_server_name = "Team16-MLFlow"
bucket_name="iti113-team16-bucket" # s3://iti113-team16-bucket

### Install dependencies and Setup

#### Session setup

Setup sagemaker and s3 session clients

In [19]:
import io
import os
import sagemaker
import boto3

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification

sagemaker_client = None
s3_client = None
try:
    sagemaker_session = sagemaker.Session()
    sagemaker_client = boto3.client("sagemaker")
    s3_bucket = sagemaker_session.default_bucket()
    s3_client = boto3.client('s3')
    s3_data_key=f"{base_folder}/data/v1/data.csv"
    s3_data_path = f"s3://{bucket_name}/{s3_data_key}"
    s3_data_dir_uri = f"s3://{bucket_name}/{base_folder}/data/v1"
    print(f"DataSet will be stored inside: {s3_data_path}")
except Exception as e:
    print(f"Error initializing SageMaker session or S3 client: {e}")
    s3_data_path = None
# minimize traceback in the output as we are not interested in the details
if not sagemaker_client or not s3_client:
    raise Exception("Failed to initialize SageMaker session or S3 client.")


DataSet will be stored inside: s3://iti113-team16-bucket/ai4i/data/v1/data.csv


Setup mlflow client

In [20]:
import mlflow
mlflow_tracking_server_arn = None
try:
    response = sagemaker_client.describe_mlflow_tracking_server(
        TrackingServerName=tracking_server_name
    )
    # ARN of MLflow Tracking Server
    mlflow_tracking_server_arn = response['TrackingServerArn']
    print(f"Found MLflow Tracking Server ARN: {mlflow_tracking_server_arn}")
except Exception as e:
    print(f"Could not find tracking server: {e}")
    mlflow_tracking_server_arn = None

# minimize traceback in the output as we are not interested in the details
if not mlflow_tracking_server_arn:
    raise Exception("Failed to find MLflow Tracking Server.")

# IAM role for SageMaker execution
role = sagemaker.get_execution_role()

print(f"SageMaker Role ARN: {role}")
print(f"MLflow Tracking Server ARN: {mlflow_tracking_server_arn}")

# Connect to the MLflow Tracking Server
# Set the MLflow tracking URI to managed server
if mlflow_tracking_server_arn:
    mlflow.set_tracking_uri(mlflow_tracking_server_arn)
    print("MLflow tracking URI set successfully.")

# Define an experiment name. If it doesn't exist, MLflow creates it.
mlflow.set_experiment(experiment_name)

print(f"MLflow tracking URI set to: {mlflow.get_tracking_uri()}")
print(f"MLflow experiment set to: '{experiment_name}'")

Found MLflow Tracking Server ARN: arn:aws:sagemaker:ap-southeast-1:837028399719:mlflow-tracking-server/Team16-MLFlow
SageMaker Role ARN: arn:aws:iam::837028399719:role/iti113-team16-sagemaker-iti113-team16-domain-iti113-team16-Role
MLflow Tracking Server ARN: arn:aws:sagemaker:ap-southeast-1:837028399719:mlflow-tracking-server/Team16-MLFlow
MLflow tracking URI set successfully.
MLflow tracking URI set to: arn:aws:sagemaker:ap-southeast-1:837028399719:mlflow-tracking-server/Team16-MLFlow
MLflow experiment set to: 'ai4i-Experiment'


#### Upload DataSet To S3

In [21]:
import pandas as pd
if s3_data_path is None:
    raise Exception("S3 data path is not set. Cannot proceed with dataset creation.")
df = pd.read_csv("ai4i2020.csv")
df.to_csv(s3_data_path, index=False)
print(f"Dataset v1.0 created and uploaded to: {s3_data_path}")

Dataset v1.0 created and uploaded to: s3://iti113-team16-bucket/ai4i/data/v1/data.csv


Let us test whether we can load the dataset

In [22]:
try:
    df_loaded = pd.read_csv(s3_data_path)
    print("Successfully loaded dataset v1.0:")
    print(df_loaded.head(1))
except Exception as e:
    print(f"An error occurred: {e}")
    print("\nPlease double-check that your bucket and folder names are correct in Step 1.")

Successfully loaded dataset v1.0:
   UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860    M                298.1                    308.6   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  TWF  \
0                    1551         42.8                0                0    0   

   HDF  PWF  OSF  RNF  
0    0    0    0    0  


### Create the SageMaker Pipeline

In [23]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, TrainingInput
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.properties import PropertyFile
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.conditions import ConditionNot
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import ParameterFloat, ParameterString
from sagemaker.model_metrics import ModelMetrics, FileSource
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

In [24]:
experiment_name_param = ParameterString(name="ExperimentName", default_value=experiment_name)
metric_threshold_param = ParameterFloat(name="F2Threshold", default_value=0.70)
pipeline_parameters = [experiment_name_param, metric_threshold_param]

#### Processing Step Definition

In [25]:
processing_instance_type = "ml.t3.medium" # cheapest $0.063/hour
preprocessor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve("sklearn", sagemaker_session.boto_region_name, "1.2-1"),
    command=[
        "python3",
    ],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="preprocess-data",
    role=role,
)

step_preprocess = ProcessingStep(
    name="PreprocessData",
    processor=preprocessor,
    inputs=[ProcessingInput(source=s3_data_dir_uri, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="preprocess.py",
)


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


#### Training Step Definition

In [26]:
training_instance_type = "ml.t3.large" # second cheapest $0.127/hour

# Training Step
sklearn_estimator = SKLearn(
    entry_point="train.py", 
    framework_version="1.2-1",
    instance_type=training_instance_type,
    role=role,
    hyperparameters={
        "tracking_server_arn": mlflow_tracking_server_arn,
        "experiment_name": experiment_name_param,
        "model_output_path": "/opt/ml/model",
    },
    py_version="py3",
    requirements="requirements.txt",
    depends_on=[step_preprocess]  # Explicitly depends on the preprocess
)

step_train = TrainingStep(
    name="TrainModel",
    estimator=sklearn_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        )
    },
)

#### Evaluation Step Defintion 

In [27]:
evaluation_processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve("sklearn", sagemaker_session.boto_region_name, "1.2-1"),
    command=['python3'],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="evaluate-model",
    role=role,
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_eval = ProcessingStep(
    name="EvaluateModel",
    processor=evaluation_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_preprocess.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")],
    code="evaluate.py",  # SageMaker will handle uploading and running this script
    job_arguments=[  # Pass arguments here instead of in command
        "--model-path", "/opt/ml/processing/model",
        "--test-path", "/opt/ml/processing/test",
        "--output-path", "/opt/ml/processing/evaluation",
        "--model-package-group-name", model_package_group_name,
        "--region", "ap-southeast-1",
    ],
    property_files=[evaluation_report],
    depends_on=[step_train]  # Explicitly depends on the train process
)


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


#### Model Registration Step

The model registration follows the following logic 
```bash
if cond_no_registered 
   step_register_new
else if cond_metric 
   step_register_better_model
end if
```
where :
- cond_no_registered check whether existing baseline model exist
- cond_metric check whether new model has higher value of metric (F2) than existing baseline model 

In [28]:
# RegisterModel step (always defined, but executed conditionally)
model_metrics_report = ModelMetrics(
    model_statistics=FileSource(
        s3_uri=step_eval.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
        content_type="application/json"
    )
)

step_register_new = RegisterModel(
    name="RegisterNewModel",
    estimator=sklearn_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics_report,
    approval_status="PendingManualApproval",
)

step_register_better_model = RegisterModel(
    name="RegisterBetterModel",
    estimator=sklearn_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics_report,
    approval_status="PendingManualApproval",
)


# Conditions: check accuracy > threshold OR no model exists
cond_metric = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="f2"
    ),
    right=metric_threshold_param
)

cond_no_registered = ConditionEquals(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="baseline_exists" # Check the key added to the report
    ),
    right=False # Condition is TRUE if baseline_exists is False
)

# Outer step: Checks for existence of registered model first
step_cond_metric = ConditionStep(
    name="CheckMetric",
    conditions=[cond_metric],
    if_steps=[step_register_better_model], # Register model if metric (F2) is high
    else_steps=[],
)

step_cond_no_registered = ConditionStep(
    name="CheckIfModelExists",
    conditions=[cond_no_registered],
    if_steps=[step_register_new], # Register model if no baseline exists
    else_steps=[step_cond_metric], # Do nothing if a model exists and F2 was low
)

#### Pipeline Creation And Execution

In [29]:
# Define steps in the pipeline
pipeline_steps = []
pipeline_steps.append(step_preprocess)
pipeline_steps.append(step_train)
pipeline_steps.append(step_eval)
pipeline_steps.append(step_cond_no_registered) 
# Define Pipeline
pipeline = Pipeline(
    name=pipeline_name,
    parameters=pipeline_parameters,
    steps=pipeline_steps
)
pipeline.upsert(role_arn=role)
execution = pipeline.start()

#### Pipeline Status and Cleanup

##### Pipeline Status Function Definition

In [45]:
def get_pipeline_status(name)->str:
    try:
        response = sagemaker_client.describe_pipeline(
            PipelineName=name
        )
        result= response["PipelineStatus"]
    except sagemaker_client.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFound':
            result= (f"Pipeline {name} not found {e}")
        else:
            result= (f"Unknown error {e.response} {type(e)}")
    return result
def print_pipeline_status(name):
    print(f"Pipeline {name} is ",get_pipeline_status(name))

##### Pipeline Cleanup Function Definition

In [42]:
def remove_pipeline(name)->str:
    try:
        response = sagemaker_client.delete_pipeline(
            PipelineName=name
        )
        print(f"Pipeline '{name}' deleted successfully.")
        print(response)
    except sagemaker_client.exceptions.ResourceNotFoundException:
        print(f"Pipeline '{name}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

##### Status check

In [47]:
# uncomment this to remove pipeline or call remove_pipeline(pipeline_name)
#pipeline.delete()
print_pipeline_status(pipeline_name)

Pipeline AI4IClassifierPipeline is  Active


## Pipeline Automation For Deployment

This pipeline will be triggered when a new model is registered.

### Deployment Pipeline Definition

In [39]:
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.parameters import ParameterString
import sagemaker
deploy_pipeline_name = "DeployModelPipeline"
# Define Parameters for the deployment pipeline
# This will be provided by the EventBridge trigger
model_package_arn_param = ParameterString(
    name="ModelPackageArn", default_value="")
role_param = ParameterString(name="ExecutionRole", default_value=role)
endpoint_name_param = ParameterString(
    name="EndpointName", default_value="AI4I-predictor-endpoint")

# Create a ScriptProcessor for deployment
# Using a more recent scikit-learn version is generally a good idea
deploy_processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve(
        "sklearn", sagemaker_session.boto_region_name, version="1.2-1"),
    command=["python3"],
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role_param,
    base_job_name="deploy-registered-model"
)

# Define the deployment step that takes the model ARN as an argument
step_deploy = ProcessingStep(
    name="DeployRegisteredModel",
    processor=deploy_processor,
    code="deploy.py",
    job_arguments=[
        "--model-package-arn", model_package_arn_param,
        "--role", role_param,
        "--endpoint-name", endpoint_name_param,
        "--region", "ap-southeast-1"
    ]
)

# Define the independent deployment pipeline
deploy_pipeline = Pipeline(
    name="DeployModelPipeline",
    parameters=[model_package_arn_param, role_param, endpoint_name_param],
    steps=[step_deploy]
)

# Create or update the pipeline definition
# Capture the response which contains the ARN
response = deploy_pipeline.upsert(role_arn=role)

# Extract the ARN from the response dictionary
pipeline_arn = response['PipelineArn']

print(f"Deployment pipeline ARN: {pipeline_arn}")

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


Deployment pipeline ARN: arn:aws:sagemaker:ap-southeast-1:837028399719:pipeline/DeployModelPipeline


In [48]:
print_pipeline_status(deploy_pipeline_name)

Pipeline DeployModelPipeline is  Active


## Deployment Pipeline Execution

In [53]:
from datetime import datetime
def start_deploy_pipeline(model_package_arn):
    deploy_pipeline_name = "DeployModelPipeline"
    endpoint_name = "AI4I-predictor-endpoint"
    now = datetime.now() # current date and time
    now_str = now.strftime("%Y-%m-%d-%H-%M-%S") # year-month-date_hour_minute_seconds e.g. 2025-08-20_11_30_05
    response = sagemaker_client.start_pipeline_execution(
        PipelineName=deploy_pipeline_name,
        PipelineExecutionDisplayName=f"{deploy_pipeline_name}-{now_str}",
        PipelineParameters=[
            {
                'Name': 'ModelPackageArn',
                'Value': model_package_arn
            }
        ], # other parameter we use default values EndpointName = AI4I-predictor-endpoint defined in our pipeline definition above
        PipelineExecutionDescription=f'Deploy image {model_package_arn} to {endpoint_name} on {now_str}'
    )
    print(response)

In [54]:
start_deploy_pipeline('arn:aws:sagemaker:ap-southeast-1:837028399719:model-package/AI4IPipelineModels/3') # change the last digit for model version i.e. /3 to /4 etc

{'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-1:837028399719:pipeline/DeployModelPipeline/execution/k2ycxmhebcnt', 'ResponseMetadata': {'RequestId': '41b0b8d3-17c6-459e-8352-1cae6cc81560', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '41b0b8d3-17c6-459e-8352-1cae6cc81560', 'content-type': 'application/x-amz-json-1.1', 'content-length': '124', 'date': 'Fri, 22 Aug 2025 03:30:11 GMT'}, 'RetryAttempts': 0}}


In [55]:
# uncomment this to remove pipeline or call remove_pipeline(deploy_pipeline_name)
#deploy_pipeline.delete()
print_pipeline_status(deploy_pipeline_name)

Pipeline DeployModelPipeline is  Active


In [36]:
import boto3
import json

# Initialize the EventBridge client
events_client = boto3.client("events")

# Define the event pattern to listen for
# This pattern triggers when a model package in your group has its status changed to "Approved"
event_pattern = {
    "source": ["aws.sagemaker"],
    "detail-type": ["SageMaker Model Package State Change"],
    "detail": {
        "ModelPackageGroupName": [model_package_group_name], # From cell 10
        "ModelApprovalStatus": ["Approved"]
    }
}

# Define the target for the rule (our deployment pipeline)
# We need to map the event's detail to the pipeline's parameters
target = {
    "Id": "DeployModelPipelineTarget",
    "Arn": pipeline_arn, # The ARN of the pipeline we just created
    # "RoleArn": role, # The execution role for the pipeline
    "SageMakerPipelineParameters": {
        "PipelineParameterList": [
            {
                # Map the ARN from the event to the pipeline's "ModelPackageArn" parameter
                "Name": "ModelPackageArn",
                "Value": "$.detail.ModelPackageArn"
            }
        ]
    }
}

# Create or update the EventBridge rule
try:
    rule_name = "AI4I-Team16-TriggerDeploymentOnApproval"
    print(f"Creating or updating EventBridge rule: {rule_name}")
    response = events_client.put_rule(
        Name=rule_name,
        EventPattern=json.dumps(event_pattern),
        State="ENABLED",
        Description="Triggers the SageMaker pipeline to deploy a churn model upon approval."
    )
    
    # Add the pipeline as a target for the rule
    events_client.put_targets(Rule=rule_name, Targets=[target])
    print("EventBridge rule created successfully!")
    print("Now, when a model is approved in the Model Registry, the deployment pipeline will trigger automatically.")

except Exception as e:
    print(f"Error creating rule: {e}")

Creating or updating EventBridge rule: AI4I-Team16-TriggerDeploymentOnApproval
Error creating rule: An error occurred (AccessDeniedException) when calling the PutRule operation: User: arn:aws:sts::837028399719:assumed-role/iti113-team16-sagemaker-iti113-team16-domain-iti113-team16-Role/SageMaker is not authorized to perform: events:PutRule on resource: arn:aws:events:ap-southeast-1:837028399719:rule/AI4I-Team16-TriggerDeploymentOnApproval because no identity-based policy allows the events:PutRule action


### Cloud Watch Logs

In [34]:
import boto3

# Enter the name of your SageMaker endpoint
endpoint_name = "AI4I-predictor-endpoint"

# The log group is created based on the endpoint name
log_group_name = f"/aws/sagemaker/Endpoints/{endpoint_name}"

# Create a CloudWatch Logs client
logs_client = boto3.client("logs")

print(f"Searching for logs in: {log_group_name}\n")

try:
    # Find all log streams in the log group, ordered by the most recent
    response = logs_client.describe_log_streams(
        logGroupName=log_group_name,
        orderBy='LastEventTime',
        descending=True
    )

    log_streams = response.get("logStreams", [])

    if not log_streams:
        print("No log streams found. The endpoint might not have processed any requests yet.")
    
    # Loop through each stream and print its recent log events
    for stream in log_streams:
        stream_name = stream['logStreamName']
        print(f"--- Logs from stream: {stream_name} ---")

        # Get log events from the stream
        log_events = logs_client.get_log_events(
            logGroupName=log_group_name,
            logStreamName=stream_name,
            startFromHead=False,  # False gets recent logs first
            limit=50  # Get up to 50 recent log events
        )
        
        # Print events in chronological order
        for event in reversed(log_events.get("events", [])):
            print(event['message'].strip())
        
        print("-" * (len(stream_name) + 24), "\n")

except logs_client.exceptions.ResourceNotFoundException:
    print(f"Error: Log group '{log_group_name}' was not found.")
    print("Please check the endpoint name and ensure it has been invoked.")
except Exception as e:
    print(f"An error occurred: {e}")

Searching for logs in: /aws/sagemaker/Endpoints/AI4I-predictor-endpoint

Error: Log group '/aws/sagemaker/Endpoints/AI4I-predictor-endpoint' was not found.
Please check the endpoint name and ensure it has been invoked.
